# Big Data for Biologists: Decoding Genomic Function- Class 5

## How can we score sequence alignments and align multiple sequences?     How do you map sequencing reads to a reference genome?

##  Learning Objectives
***Students should be able to***
 <ol>
 <li><a href=#ScoreAlignments>Use DNA alignment scoring matrices to select the best alignment of two sequences from two possible alignments</a></li>
 <li><a href=#AlignMuscle>Align multiple sequences using modules from the BioPython package</a></li> 
 <li><a href=#MapSeqReads>Map sequence reads to the human reference genome </a></li>
 <li><a href=#Qualityscores>Interpret quality scores from the output of a read mapping and alignment algorithm</a></li>
 </ol>

## Use DNA alignment scoring matrices to select the best alignment of two sequences from two possible alignments <a name='ScoreAlignments' />

One of the outputs from the pairwise sequence alignment that we ran in the previous class is the sequence alignment score. 

Different algorithms have different ways of calculating the sequence aligmment score, but most are based on scoring substitutions and gaps (ie. insertions or deletions) in the sequences. 

We are going to look at a simple example now of a sequence alignment to learn how the scores are generated. 

If we have two potential alignments between sequences, how can we tell which alignment is better? 
For example, given the sequences: 

<img src="../Images/5-AlignSeq1.png" style="width: 12%; height: 10%" align="center"/>
and 

<img src="../Images/5-AlignSeq2.png" style="width: 11%; height: 9%" align="center"/>


we can produce two different alignments: 

<img src="../Images/5-EditDistance1.png" style="width: 14%; height: 30%" align="center"/>

and 

<img src="../Images/5-EditDistance2.png" style="width: 15%; height: 35%" align="center"/>


One way to determine which alignment is better is to consider the number of edits (changes) that must be made to the sequence to make it match another sequence. 

For example, the edit distance between TGTTACGG and GGTTACGG is 1: 

<img src="../Images/5-EditDistance-nucleotides-1.png" style="width: 70%; height: 70%" align="center"/>


The edit distance between TGTTACGG and GGTTAC-G is 2:

<img src="../Images/5-EditDistance-nucleotides-2.png" style="width: 75%; height: 75%" align="center"/>

**Thought questions** 

What is the edit distance between "TG-TT-ACGG" and "-GGTTGACTA" in the alignment above?

**Your answer here**: 

What is the edit distance between "TGTT-ACGG" and "GGTTGACTA"? 

**Your answer here**: 

Which is the better alignment?

**Your answer here**: "TG-TT-ACGG" and "-GGTTGACTA"

Using the edit distance between two aligned sequences to measure the quality of the alignment is a good first step. 

However, not all alignment errors carry the same weight. For DNA, the bases A and G are more similar in structure than the bases C and T. A and G are known as purines, whereas C and T are known as pyrimidines. 

<img src="../Images/5-BasePairs_Purines_and_Pyrimidines_Updated.png" style="width: 70%; height: 80%" align="center"/>

Additionally, alignments with many insertions/deletions are considered poor quality, so most aligners will place a heavier penalty on a gap in the alignment (indicate by "\_") than on a mismatch. A typical set of alignment scores is summarized in the matrix below: 

<img src="../Images/5-SequenceScoringMatrix.png" style="width: 30%; height: 60%" align="center"/>

The matrix assumes that a base in one of the rows is aligned with a base in one of the columns. If the bases are the same (i.e. A aligns with A, T aligns with T, etc.) the alignment score is positive. However, in the case of a mismatch or an insertion/deletion, the alignment score is negative. We sum the score across all bases in the alignment to determine an overall score for the alignment. 

How would you create a dictionary to store the alignment scores in the matrix above? 

**Task**: Write code to store scores in the first row ('A') of the matrix. 

In [ ]:
DNAdict={} 
## BEGIN SOLUTION ## 
## END SOLUTION ## 

In [ ]:
DNAdict={}
#Iterate through each row in the Transition/Transversion matrix 
for row in ['A','T','C','G','-']:
    #create a dictionary to store the columns 
    DNAdict[row]=dict() 
    for column in ['A','T','C','G','-']: 
        #As a default, assume the maximum penalty score -- i.e. a poor match. 
        DNAdict[row][column]=-5 

     
#Account for insertion/deletion penalties 
DNAdict['-']['A']=-7
DNAdict['-']['T']=-7
DNAdict['-']['C']=-7
DNAdict['-']['G']=-7

DNAdict['A']['-']=-7
DNAdict['T']['-']=-7
DNAdict['C']['-']=-7
DNAdict['G']['-']=-7


#Correct the scores that are greater than -5 in the matrix 

#Fill in the dictionary values to indicate that matching a base correctly earns a score of 1 
## BEGIN SOLUTION ## 
## END SOLUTION ## 

#Fill in the dictionary values to indicate that aligning a purine to another purine gets a score of -1. 
## BEGIN SOLUTION 
## END SOLUTION 

#Fill in the dictionary values to indicate that aligning a pyrimidine to another pyrimidne gets a score of -1. 
## BEGIN SOLUTION ## 
## END SOLUTION ## 

print(DNAdict)

Using the scores encoded in DNAdict, fill in the code below to score two DNA alignments

In [ ]:
def score_alignment(sequence1,sequence2,DNAdict): 
    alignment_score=0
## BEGIN SOLUTION ## 
    for i in range(0,len(sequence1)):
        cur_base_1=sequence1[i] 
        cur_base_2=sequence2[i]
        score= ##Fill in the score 
        alignment_score+=score 
    return alignment_score 
## END SOLUTION ## 

Run the score_alignment function you wrote above to check if you get the expected alignment scores for our example:

In [ ]:
alignment1_score=score_alignment('TG-TT-ACGG','-GGTTGACTA',DNAdict)
print("alignment1_score:"+str(alignment1_score)) #should be -22
alignment2_score=score_alignment('TGTT-ACGG','GGTTGACTA',DNAdict)
print("alignment2_score:"+str(alignment2_score)) #should be -13

## Align multiple sequences using modules from the BioPython pacakge and the MUSCLE algorithm<a name='AlignMuscle' />

If we want to compare three or more sequences together, for example, the two insulin sequences from mouse and the insulin sequence from human that we were looking at in the last class, we can perform a multiple sequence alignment. 

Today we are just going to use the example of looking at three sequences, but we will see more examples later where multiple sequence alignments are particularly useful for looking at how well sequences are conserved across species or within populations. 

For the multiple sequence alignments today, we are going to be using an algorithm called MUSCLE. We have preinstalled MUSCLE on the Jupyter Notebook server that we are using. Documentation or information on downloading the program an be found on the [MUSCLE program website](http://www.drive5.com/muscle/). 

The input for the multiple sequence alignment is a list of sequences in FASTA format. We have created a file in 
data/Human and Mouse Insulin Genes.fa. Which you can see below. 

In [ ]:
# Runs a multiple sequence alignment between the Human-Insulin Gene NG_007114.1
# the Mouse Insulin GeneID 16333 and the Mouse Insulin GeneID 16334

# we are importing the MuscleCommandline here from Bio.Align.Applicaitons 
from Bio.Align.Applications import MuscleCommandline

# defines a variable with the path of the executable for the MUSCLE algorithm program 
muscle_exe="/usr/bin/muscle"

#runs the multiple sequence alignment and writes to an output file in ClustalW format 
muscle_cline = MuscleCommandline(muscle_exe,input="data/Human and Mouse Insulin Genes.fa",out="Human_and_Mouse_Insulin_Genes.aln",clw=True)
muscle_cline()

In [ ]:
#use the 'cat' command to view file contents. 
!cat Human_and_Mouse_Insulin_Genes.aln

Thought questions:

* What does your analysis tell you about the similarities between human and mouse insulin genes? 
* Which insulin gene from mouse is more similar to the human insulin gene? 

## Mapping and aligning  sequencing reads to a reference genome<a name='MapSeqReads' />

We are going to look now at the question of how do you map and align sequencing reads (the output of DNA sequencing experiments) to the reference human genome. 

You might be wondering what the difference is between mapping and alignment. 

The output of DNA sequencing machines is typically millions of short sequencing reads (<200 base pairs). They are often referred to as ** short-reads**. 

Since the human genome has 3 billion base pairs, running sequence alignments at each position for every sequencing read is too slow.   

Many popular alignment algorithms, such as the [Bowtie2 algorithm](http://bowtie-bio.sourceforge.net/bowtie2/manual.shtml#local-alignment-example) that we will use in this example, begin the alignment task with a mapping phase. It is important to draw the distinction between **mapping** and **alignment** : 

<img src="../Images/5-MappingVsAlignment.png" style="width: 50%; height: 50%" align="center"/>



We are going to look now at an example of the Bowtie2 mapping and alignment algorithm. First, we need to prepare our data. 

We will be using sequences from a sample of human blood. The FASTA reads from the sequencer are in the file samples.fasta. This file contains a small snippet of the actual sequencing file for illustrative purposes; the full file generated by the sequencer generally contains millions of reads. 

In [ ]:
#The data/sample.fasta file contains the 11 sequences that we will align to the reference. Use the 'cat' command to examine the contesnts of this file. 
! cat data/samples.fasta

We want to check where these sequences map along the human genome. Since the human genome contains 3 billion bases, the alignment process can take several hours, so we are going to abbreviate it and only check how many of the sequences align to the first 1000 lines of the human reference genome file. 

In [ ]:
#Use the 'head' command to examine the first 1000 lines of the human reference genome 
!head -n1000 /data/hg19.genome.fa

The beginning of the hg19.genome.fa contain only 'N' characters -- this indicates that this portion of the genome was not sequenced successfully. The use of N characters to indicate non-sequenced portions of the genome is referred to as "hard masking". We can use the code below to obtain the first 1000 lines of the reference genome sequence that are not composed of 'N''s. 

In [ ]:
#find the first line not composed of 'N' in the reference genome file 
#open the hg19.genome.fa reference file 
reference=open("/data/hg19.genome.fa",'r')
#store the chromosome header 
header=reference.readline()
print(header)
line_index=0 
cur_line=reference.readline() 
while cur_line.startswith('N'): 
    line_index+=1 
    cur_line=reference.readline() 
print(line_index)

In [ ]:
#Extract the lines 1 - 1201 from the reference fasta file to make 
#sure we have 1000 lines of data that do not start with N. 
!head -n 1201 /data/hg19.genome.fa > reference.fasta 

Note that the > sign in the line of code takes the output of the first part of the command and 'pipes' (or sends) it to an output file called reference.fasta.

Great, now we have the data ready and can perform the mapping and alignment algorithms.

**Thought question**: 

If you had to come up with an algorithm to map a DNA sequence to the reference genome, how would you proceed? It might help to think of the task as similar to finding a specific word in a book -- what are some shortcuts you might use to avoid having to read each page in the book? 

To map sequencing reads to a reference genome, Bowtie2, and other algorithms create a compressed genome index. This index is a map of where each short subsequence of length 'k' hits the genome. A popular approach is known as **hash-based mapping**. In the figure below, we break up the sequence CATGGTCATTGGTTCC into overlapping windows of length 3 (k=3). We then tally the positions along this sequence where each window can be found: 
<font size=1>(figure borrowed with permission from Aaron Quinlan)</font>


<img src="../Images/5-HashBasedMappingStep1.png" style="width: 70%; height: 80%" align="center"/>


We now create an index for the first 1201 bases of the reference genome that we stored in the file reference.fasta 
Run the **bowtie_index** function from the helper file **"alignment.py"** to do this. 

In [ ]:
#Import the alignment helper functions 
#import the sequence_alignment_helpers.py file in the helpers directory
import sys
sys.path.append('../helpers')

import alignment 
from alignment import * 


bowtie_index("reference.fasta","reference")


This creates a number of index files in our current working directory, all beginning with the "reference" prefix and ending with "bt2" (abbreviation for bowtie2). 

In [ ]:
#Use the ls command to see the new index files that were generated by the bowtie_index command. 
!ls 

Now that we have built the index for the reference, we can use it to map reads. The kmer index is used to quickly find candidate alignment locations in the genome. As an example, we look at the first three bases of an example read TGGTCA. We find that the kmer TGG appers in the index at positions 3, 10. 

<img src="../Images/5-HashStep2.png" style="width: 100%; height: 90%" align="center"/>


This approach works great when there are no errors in the sequence reads. However, often we may encounter errors in the genetic sequence. 
This can be due to either: 

* errors in sequencing 

* genetic variants 

We must allow for errors such as insertions, deletions, and mismatches when generating sequence alignments. 

For example, if there is a mutation in our toy sequence where the last base is changed from an A to a T, we will no longer be able to find a match in the reference index. Hence, there is some ambiguity whether the sequence should map to position 3 or position 10 in the reference. 
<img src="../Images/5-MatchAmbiguity.png" style="width: 100%; height: 100%" align="center"/>

In cases like this, we must fall-back on the alignment techniques discussed at the beginning of the tutorial -- We score alignments of the sequence to the reference at both positions 3 and 10, and we pick the alignment that maximizes the score. 


We use the helper function **align** to run the Bowtie2 algorithm that will map and align our sample.fasta file of reads to the reference genome, using the Bowtie index that we generated above. 

We then slide along the sequence, checking the index location of the next kmer (TCA). We find that the index has TCA at position 6. This indicates that '10' is no longer a possible position for the first kmer, so we exclude it from the list of hash matches. 

<img src="../Images/5-HashStep3.png" style="width: 100%; height: 90%" align="center"/>

In [ ]:
align("samples.fasta","reference","aligned")

The **align** function generates an output bed file called aligned.bed. Let's examine the contents of this file: 

In [ ]:
!cat aligned.bed 

We observe that 6 of the 11  reads map to more than one position inthe reference genome. This is because more than half of the human genome is comprised of repeats. For example, at the beginning of chromosome 1 we see many instances of the sequence taccc. Note that lower-case letters are often used in a FASTA reference sequence to indicate repetitive or low-complexity regions. This practice is called "soft masking" (in contrast, recall that the use of "N" to indicate non-sequenced regions of the genome is referred to as "hard masking"). 


<img src="../Images/5-Repeats.png" style="width: 50%; height: 50%" align="center"/>

We see the sequences from samples.fasta listed in the fourth column. 

* The first column contains the name of the chromosome on reference.fasta along which the read aligned. 

* The second and third columns contain, respectively, the start and end coordinates of the aligned read along the reference. 

* The sixth column can contain strand information -- all our reads align to the positive strand of the reference. 

* The fifth column contains the mapping quality, which is a number between 0 and 255. Higher numbers indicate better 
mapping to the reference. Notice that this value is 42 for the four reads that mapped to only a single location 
in the reference sequence, and 1 for the 6 reads that mapped to multiple locations in the reference. It is 0 for 
the read that did not map to the reference. 

## Interpret quality scores from the output of a read mapping and alignment algorithm <a name='Qualityscores' />

The quality score give a sense of how significant the alignment is. 

It is calculated as : 

**-10\*log10(1 - (1/number of possible alignments))** 

For example, in the case of a unique alignment to the reference: 

number of possible alignments = 1 
```
score = -10*log10(1-1/1)
score = -10*log10(0) 
score = 255
```
Since the log of 0 is not defined, the score should be infinite. However, scores are capped at 255 to avoid the inconvenience of dealing with infinite numbers. 

If there are two possible alignments: 
```
score=-10*log10(1-1/2)
score=-10*log10(0.5) 
score= 3.01 
```
If there are four possible alignments: 
```
score=-10*log10(1-1/4)
score=-10*log10(0.75) 
score= 1.24 
```
In the extreme case, if there are 100 possible alignments: 
```
score=-10*log10(1-1/100)
score=-10*log10(0.99) 
score= 0.0436 
```
If the sequence does not map to the reference genome, the alignment score is set to 0. 
From the examples above, we observe that the more possible alignments there are for the sequence in the reference genome, the lower the alignment score. 



